In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        #print(f"d_model : {d_model}")
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        
    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output
        
    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)
        
    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)
        
    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))
        
        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        
        pe = torch.zeros(max_seq_length, d_model)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        self.register_buffer('pe', pe.unsqueeze(0))
        
    def forward(self, x):
        #print((x + self.pe[:, :x.size(1)]).shape)
        return x + self.pe[:, :x.size(1)]

class CustomWeightLayer(nn.Module):
    def __init__(self, input_dim, max_tokens):
        super(CustomWeightLayer, self).__init__()
        self.input_dim = input_dim
        self.max_tokens = max_tokens
        self.expanded_output_dim = 2 * self.max_tokens - 1

        # [max_tokens, 2*max_tokens-1] 크기의 가중치 행렬
        self.weight = nn.Parameter(torch.Tensor(self.input_dim, self.expanded_output_dim, self.max_tokens))
        self.bias = nn.Parameter(torch.Tensor(self.input_dim, self.expanded_output_dim))

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        nn.init.constant_(self.bias, 0.1)
        # 가중치 행렬의 특정 부분을 0으로 설정
        mask = torch.ones_like(self.weight)
        for i in range(self.input_dim):
            mask[i, :i] = 0  # 시작 부분을 0으로
            mask[i, i+self.input_dim:] = 0  # 끝 부분을 0으로
        self.weight.data *= mask

    def forward(self, x, curr_tokens):
        # x: [batch, max_tokens, input_dim]
        batch_size, _, _ = x.size()

        # [batch, input_dim, max_tokens]로 transpose
        x = x.transpose(1, 2)

        # [batch, input_dim, 2*max_tokens-1]를 얻기 위한 연산
        output = torch.empty(batch_size, self.input_dim, self.expanded_output_dim, device=x.device)
        for i in range(self.input_dim):
            output[:, i, :] = F.linear(x[:, i, :], self.weight[i], self.bias[i])

        # [batch, input_dim, :2*curr_tokens-1] 범위 사용
        output = output[:, :, :2*curr_tokens-1]

        # 결과를 [batch, input_dim, curr_tokens]로 조정
        final_output = torch.empty(batch_size, self.input_dim, curr_tokens, device=x.device)
        for i in range(curr_tokens):
            start_idx = i
            end_idx = start_idx + curr_tokens
            final_output[:, :, i] = output[:, :, start_idx:end_idx].sum(dim=2)

        # 최종 결과를 [batch, curr_tokens, input_dim]으로 transpose
        final_output = final_output.transpose(1, 2)
        # [batch, curr_tokens, input_dim]에서 [batch, max_tokens, input_dim]으로 변경하기 위해 패딩 추가
        padding_size = self.max_tokens - curr_tokens
        if padding_size > 0:
            final_output = F.pad(final_output, (0, 0, 0, padding_size), "constant", 0)
        
        return final_output

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout, max_seq_length):
        super(EncoderLayer, self).__init__()
        self.custom = CustomWeightLayer(d_model, max_seq_length)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, length):
        #print(f"X: {x}, mask: {mask}")
        attn_output = self.custom(x, length)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [4]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout, max_seq_length) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_emb = self.encoder_embedding(src)
        tgt_emb = self.decoder_embedding(tgt)
        src_lengths = torch.sum(src_emb != 0, dim=1)
        tgt_lengths = torch.sum(tgt_emb != 0, dim=1)

        #print("Source lengths:", src_lengths, "Max Length:", src_emb.shape)
        #print("Target lengths:", tgt_lengths)
        src_embedded = self.dropout(self.positional_encoding(src_emb))
        tgt_embedded = self.dropout(self.positional_encoding(tgt_emb))
        
        enc_output = src_embedded
        #print(src_embedded.shape)
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, 100)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = 100
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout).to(device)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(10):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")
print(transformer)

Epoch: 1, Loss: 8.685918807983398
Epoch: 2, Loss: 8.563715934753418
Epoch: 3, Loss: 8.478215217590332
Epoch: 4, Loss: 8.409462928771973
Epoch: 5, Loss: 8.348128318786621
Epoch: 6, Loss: 8.277074813842773
Epoch: 7, Loss: 8.199198722839355
Epoch: 8, Loss: 8.113490104675293
Epoch: 9, Loss: 8.029463768005371
Epoch: 10, Loss: 7.948812484741211
Transformer(
  (encoder_embedding): Embedding(5000, 512)
  (decoder_embedding): Embedding(5000, 512)
  (positional_encoding): PositionalEncoding()
  (encoder_layers): ModuleList(
    (0-5): 6 x EncoderLayer(
      (custom): CustomWeightLayer()
      (feed_forward): PositionWiseFeedForward(
        (fc1): Linear(in_features=512, out_features=2048, bias=True)
        (fc2): Linear(in_features=2048, out_features=512, bias=True)
        (relu): ReLU()
      )
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (d

In [5]:
from transformers import BertModel, BertTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/tokenizer/")
model = BertModel.from_pretrained("google-bert/bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/model/")


In [65]:
text = "Hi ","my name"
encoded = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=256)
input_ids = encoded["input_ids"]

input_ids.shape

torch.Size([2, 4])

In [66]:
a = [1,2,3,4]
a[-3:]

[2, 3, 4]

In [127]:
from transformers import BertModel, BertTokenizer,BertConfig
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#tokenizer = BertTokenizer.from_pretrained("bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/tokenizer/")
#model = BertModel.from_pretrained("google-bert/bert-base-cased", cache_dir="../../../data2/jeongseokoh/hub/model/")
model.to(device)
config = BertConfig.from_pretrained('google-bert/bert-base-cased', cache_dir="../../../data2/jeongseokoh/hub/model/")
text = "Hi ","my name"
encoded = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt", max_length=256)
input_ids = encoded["input_ids"].to(device)
non_zero_token_counts = (input_ids != 0).sum(dim=1).to(device)
print(f"Actual_length: {non_zero_token_counts}")
emb = model.embeddings(input_ids)
print(f"Embedding shape: {emb.shape}")

class CustomWeightLayer(nn.Module):
    def __init__(self, d_model, tokens, curr_length):
        super(CustomWeightLayer, self).__init__()
        self.d_model = d_model
        self.tokens = tokens
        self.curr_length = curr_length
        self.out_dim1 = 2 * self.tokens - 1
        self.out_dim2 = self.d_model
        self.relu = nn.ReLU()

        # 가중치 및 바이어스 초기화를 더 효율적으로 수행
        self.expand_w = nn.Parameter(torch.randn(self.out_dim1, self.tokens) * 0.01)
        self.expand_bias = nn.Parameter(torch.zeros(self.out_dim1))
        self.feature_w = nn.Parameter(torch.randn(self.out_dim2, self.d_model) * 0.01)
        self.feature_bias = nn.Parameter(torch.zeros(self.out_dim2))
        self.shrink_w = nn.Parameter(torch.randn(self.tokens, self.out_dim1) * 0.01)
        self.shrink_bias = nn.Parameter(torch.zeros(self.tokens))

        # 가중치에서 특정 패턴을 0으로 설정
        self._init_weights()

    def _init_weights(self):
        # expand_w와 shrink_w의 특정 패턴을 0으로 초기화
        with torch.no_grad():
            for i in range(self.tokens):
                for j in range(self.tokens - 1):
                    self.expand_w.data[i - self.tokens + 1 + j, i] = 0.0
                    self.shrink_w.data[i, i - self.tokens + 1 + j] = 0.0

    def forward(self, x, attention_mask=None, head_mask=None, encoder_hidden_states=None, encoder_attention_mask=None, past_key_value=None, output_attentions=None, use_cache=None):
        # 텐서 조작을 최소화하고, 불필요한 전치를 피함
        x = F.linear(x.transpose(1, 2), self.expand_w, self.expand_bias).transpose(1, 2)
        x = F.linear(x, self.feature_w, self.feature_bias)
        x = self.relu(x)
        x = F.linear(x.transpose(1, 2), self.shrink_w, self.shrink_bias).transpose(1, 2)

        return x
class CustomBertModel(BertModel):
    def __init__(self, config, d_model, tokens, curr_length):
        super(CustomBertModel, self).__init__(config)  # config 객체를 전달합니다.
    
        # BERT 모델의 어텐션을 커스텀 어텐션으로 교체
        for i, layer in enumerate(self.encoder.layer):
            self.encoder.layer[i].attention.self = CustomWeightLayer(d_model, tokens, curr_length)
print(f"Attention output: {model.encoder.layer[0].attention(emb)[0].shape}")

batch_size, tokens, feature_size = emb.size()
#print(f"batch: {batch_size}, tokens: {tokens}, feature_size: {feature_size}")
#attn = model.encoder.layer[0].attention(emb)
our_model = CustomWeightLayer(feature_size, tokens, non_zero_token_counts).to(device)
output = our_model(emb)
print(f"Ours output_shape: {output.shape}")
model.encoder.layer[0].output(output)
print(f"Ours output_shape: {output.shape}")
#output = model.encoder.layer[0].output()


Actual_length: tensor([3, 4], device='cuda:0')
Embedding shape: torch.Size([2, 256, 768])
Attention output: torch.Size([2, 256, 768])
Ours output_shape: torch.Size([2, 256, 768])


TypeError: BertOutput.forward() missing 1 required positional argument: 'input_tensor'

In [106]:
import torch

# 예제 데이터 생성: [batch, tokens, features] = [2, 8, 768]
example_input = torch.rand(2, 8, 768)  # 무작위 데이터

# 1단계: Transpose -> [batch, features, tokens]
transposed_input = example_input.transpose(1, 2)  # 또는 example_input.permute(0, 2, 1)

# 2단계: 가중치 텐서 생성 -> [tokens, 2*tokens-1] = [8, 15]
# 여기서는 예제로, 실제로 이러한 연산을 수행하기 위해서는 추가적인 커스텀 연산이 필요할 수 있습니다.
# 가중치 텐서를 무작위로 초기화합니다. 실제 사용 시에는 이 가중치를 학습할 수 있도록 설정해야 합니다.
weight_tensor = torch.rand(8, 15)  # 무작위 가중치 텐서

# 3단계: 가중치 텐서와의 곱 -> [batch, features, 2*tokens-1]
# 이 단계는 실제 행렬 곱셈 연산과 다를 수 있으며, 적절한 차원 변환을 위한 추가 작업이 필요할 수 있습니다.
# 예제 코드는 이론적 설명을 위한 것이며, 실제 구현에는 커스텀 로직이 필요할 수 있습니다.
output = torch.matmul(transposed_input, weight_tensor)

# 결과 확인
output_shape = output.shape
output_shape

torch.Size([2, 768, 15])

In [107]:
import torch

# 예제 데이터 생성: [batch, tokens, features] = [2, 256, 768]
example_input = torch.rand(2, 256, 768)  # 무작위 데이터

# 1단계: Transpose -> [batch, features, tokens]
transposed_input = example_input.transpose(1, 2)  # 또는 example_input.permute(0, 2, 1)

# 2단계: 가중치 행렬 생성 -> [tokens, 2*tokens-1] = [256, 511]
weight_matrix = torch.rand(256, 511)  # 무작위 가중치 행렬

# 3단계: 가중치 행렬과의 곱 -> [batch, features, 2*tokens-1]
output = torch.matmul(transposed_input, weight_matrix)

# 결과 확인
output_shape = output.shape
output_shape

torch.Size([2, 768, 511])

In [114]:
# 계산량 비교를 위한 파라미터 설정
seq_len = 8  # 시퀀스 길이
d_model = 768  # 임베딩 차원

# Attention 계산량
attention_calc = 3 * seq_len * d_model**2 + seq_len**2 * d_model

# 사용자 정의 가중치 연산의 계산량
custom_weight_calc = seq_len * (100 * seq_len - 1) * d_model

attention_calc, custom_weight_calc

(14204928, 4909056)